# Imports

In [1]:
from google.cloud import storage
import pandas as pd
import numpy as np
import re
import datetime
from unidecode import unidecode

# Carregando Dados

In [2]:
BUCKET_NAME = 'busca-apartamentos-bucket'
FILE = '2024-02-18 - apartamentos - apolar.csv'
TEMP_FILE = 'local.csv'

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

blop = bucket.blob(blob_name= FILE).download_as_string()
with open(TEMP_FILE, 'wb') as f:
    f.write(blop)

df = pd.read_csv('local.csv')

# Tratando Dados

In [3]:
df['atributos'].unique()

array(['203 m²', '250 m², 1 banheiro, 3 vagas, 3 quartos, pet',
       '330 m², 2 vagas, 4 quartos',
       '163 m², 2 banheiros, 2 vagas, 2 quartos, pet', nan,
       '118 m², 2 banheiros, 1 vaga, 3 quartos',
       '89 m², 2 banheiros, 1 vaga, 2 quartos',
       '103 m², 2 banheiros, 1 vaga, 3 quartos, pet',
       '90 m², 1 banheiro, 1 vaga, 2 quartos',
       '146 m², 2 banheiros, 1 vaga, 3 quartos',
       '58 m², 1 banheiro, 1 quarto', '71 m², 2 banheiros, 3 quartos',
       '97 m², 2 banheiros, 1 vaga, 2 quartos, pet',
       '108 m², 2 banheiros, 3 quartos',
       '122 m², 2 banheiros, 1 vaga, 3 quartos',
       '105 m², 2 vagas, 3 quartos',
       '108 m², 2 banheiros, 1 vaga, 3 quartos, pet',
       '143 m², 2 vagas, 3 quartos',
       '173 m², 2 banheiros, 2 vagas, 3 quartos',
       '154 m², 2 banheiros, 1 vaga, 3 quartos',
       '126 m², 2 vagas, 2 quartos',
       '115 m², 2 banheiros, 1 vaga, 3 quartos, pet',
       '156 m², 2 banheiros, 1 vaga, 3 quartos',
       '149

## Limpeza

In [6]:
df['titulo'] = df['titulo'].str.strip()
df['endereco'] = df['endereco'].str.strip()
df['descricao'] = df['descricao'].str.strip().str.replace('  ', '').str.replace('\n',' ')

# Criação de Features

In [7]:
def busca_substring(substring, string_list):
    result = np.nan
    for s in string_list:
        if substring in s:
            try:
                result = re.findall(r'\s(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?)', s)[0]
            except:
                result = s
            break
            
    return result

def separa_valores_imovel(string):

    # Padrao regex para encontrar nome e valor monetário
    padrao = r'(\w+)\sR\$\s(\d{1,3}(?:\.\d{3})*(?:,\d{2})?)'

    # Encontrar todas as correspondências na string
    correspondencias = re.findall(padrao, string)

    # Imprimir os resultados
    list_values = []
    for correspondencia in correspondencias:
        nome, valor = correspondencia
        list_values.append(f'{nome}: {valor}')
    
    return list_values

In [8]:
# Localidade
df['bairro'] = df['endereco'].apply(lambda x: x if isinstance(x,float) else x.split(', ')[2].split(' - ')[0])
df['cidade'] = df['endereco'].apply(lambda x: x if isinstance(x,float) else x.split(', ')[2].split(' -')[1])

# Atributos do imóvel
df['area'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('m²', x.split(', ')))
df['banheiros'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('banheiro', x.split(', ')))
df['suite'] = df['descricao'].apply(lambda x: x if isinstance(x, float) else 'Sim' if 'suite' in unidecode(x.lower()) else 'Não')
df['vagas_garagem'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('vaga', x.split(', ')))
df['quartos'] = df['atributos'].apply(lambda x: x if isinstance(x,float) else busca_substring('quarto', x.split(', ')))

df['area'] = df['area'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')
df['banheiros'] = df['banheiros'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')
df['vagas_garagem'] = df['vagas_garagem'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')
df['quartos'] = df['quartos'].apply(lambda x: np.nan if isinstance(x, float) else x.split(' ')[0]).astype('float64')

# Valores
df['aluguel'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('Aluguel',separa_valores_imovel(x)))
df['condominio'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('Condomínio',separa_valores_imovel(x)))
df['seguro_incendio'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('Incêndio',separa_valores_imovel(x)))
df['iptu'] = df['valores'].apply(lambda x: x if isinstance(x,float) else busca_substring('IPTU',separa_valores_imovel(x)))

df['aluguel'] = df['aluguel'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')
df['condominio'] = df['condominio'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')
df['seguro_incendio'] = df['seguro_incendio'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')
df['iptu'] = df['iptu'].apply(lambda x: x if isinstance(x,float) else x.replace(',00', '').replace('.','').replace(',','.')).astype('float64')
df['valor_total'] = df['aluguel'] + df['condominio'] + df['seguro_incendio'] + df['iptu']

# Detalhes do imóvel/condomínio
df['mobiliado'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'mobiliado' in unidecode(x.lower()) else 'Não')
df['piscina'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'piscina' in unidecode(x.lower()) else 'Não')
df['academia'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'academia' in unidecode(x.lower()) else 'Não')
df['sacada'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'sacada' in unidecode(x.lower()) else 'Não')
df['churrasqueira'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'churrasqueira' in unidecode(x.lower()) else 'Não')
df['salao_de_festas'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'salao de festas' in unidecode(x.lower()) else 'Não')


In [9]:
df['salao_de_festas'] = df['descricao'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'salao de festas' in unidecode(x.lower()) else 'Não')

# Save File

In [13]:
columns_selected = [
 'site',
 'link',
 'data_coleta',
 'titulo',
 'endereco',
 'atributos',
 'descricao',
 'bairro',
 'cidade',
 'area',
 'banheiros',
 'vagas_garagem',
 'quartos',
 'aluguel',
 'condominio',
 'seguro_incendio',
 'iptu',
 'mobiliado',
 'piscina',
 'academia',
 'sacada',
 'churrasqueira',
 'salao_de_festas'
 ]

In [32]:
df['descricao'].apply(lambda x: x if isinstance(x, float) else 'suite' in unidecode(x.lower())).sum()

116

In [15]:
df.loc[11,'link']

'https://www.apolar.com.br/alugar/curitiba/centro/alugar-residencial-apartamento-curitiba-centro-106053'

In [14]:
df.loc[df['suite'] == 'Não']

,site,data_coleta,titulo,link,endereco,valores,atributos,descricao,bairro,cidade,...,condominio,seguro_incendio,iptu,valor_total,mobiliado,piscina,academia,sacada,churrasqueira,salao_de_festas
0,Apolar,2024-02-18,Apartamento Residencial para Locação no Água V...,https://www.apolar.com.br/alugar/curitiba/agua...,"Avenida Silva Jardim, 2733, Água Verde - Curitiba","Aluguel R$ 20.000,00 Seguro Incêndio R$ 62,56T...",203 m²,Apartamento comno Bairro Água Verde Imóvel com...,Água Verde,Curitiba,...,NaN,62.56,NaN,NaN,Não,Não,Não,Não,Não,Não
11,Apolar,2024-02-18,Apartamento Residencial para Locação no Centro...,https://www.apolar.com.br/alugar/curitiba/cent...,"Rua Riachuelo, 110, Centro - Curitiba","Aluguel R$ 3.950,00 Seguro Incêndio R$ 27,43To...","58 m², 1 banheiro, 1 quarto",Apartamento com 1 quarto no Bairro Centro Imóv...,Centro,Curitiba,...,NaN,27.43,NaN,NaN,Não,Não,Sim,Sim,Não,Não
27,Apolar,2024-02-18,Apartamento Residencial para Locação no Bigorr...,https://www.apolar.com.br/alugar/curitiba/bigo...,"Alameda Júlia Da Costa, 2590, Bigorrilho - Cur...","Aluguel R$ 3.000,00 Condomínio R$ 2.000,00 Seg...","108 m², 1 vaga, 2 quartos",Apartamento com 2 quartos no Bairro Bigorrilho...,Bigorrilho,Curitiba,...,2000.00,39.14,NaN,NaN,Não,Não,Não,Sim,Sim,Não
30,Apolar,2024-02-18,Apartamento Residencial para Locação no Sítio ...,https://www.apolar.com.br/alugar/curitiba/siti...,"Rua Astorga, 680, Sítio Cercado - Curitiba","Aluguel R$ 2.940,00 Condomínio R$ 200,00 Segur...","100 m², 2 banheiros, 1 vaga, 2 quartos",Apartamento com 2 quartos no Bairro Sítio Cerc...,Sítio Cercado,Curitiba,...,200.00,36.80,NaN,NaN,Não,Não,Não,Não,Não,Não
35,Apolar,2024-02-18,Apartamento Residencial para Locação no Ecovil...,https://www.apolar.com.br/alugar/curitiba/ecov...,"Rua Professor Pedro Viriato Parigot de Souza, ...","Aluguel R$ 2.700,00 Condomínio R$ 525,00 Segur...","97 m², 1 vaga, 2 quartos",Apartamento com 2 quartos no Bairro Ecoville I...,Ecoville,Curitiba,...,525.00,36.80,NaN,NaN,Não,Sim,Não,Sim,Sim,Sim
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,Apolar,2024-02-18,Apartamento Residencial para Locação no Alto B...,https://www.apolar.com.br/alugar/curitiba/alto...,"Rua Capitão Amin Mosse, 101, Alto Boqueirão - ...","Aluguel R$ 700,00 Seguro Incêndio R$ 26,26Tota...","50 m², 1 banheiro, 1 quarto, pet",Apartamento com 1 quarto no Bairro Alto Boquei...,Alto Boqueirão,Curitiba,...,NaN,26.26,NaN,NaN,Não,Não,Sim,Não,Não,Não
358,Apolar,2024-02-18,Apartamento Residencial para Locação no Uberab...,https://www.apolar.com.br/alugar/curitiba/uber...,"Rua Alípio Schamne, 313, Uberaba - Curitiba","Aluguel R$ 700,00 Seguro Incêndio R$ 26,26Tota...","35 m², 1 banheiro, 1 quarto",Apartamento com 1 quarto no Bairro Uberaba Imó...,Uberaba,Curitiba,...,NaN,26.26,NaN,NaN,Não,Não,Não,Não,Não,Não
359,Apolar,2024-02-18,Apartamento Residencial para Locação no Barrei...,https://www.apolar.com.br/alugar/curitiba/barr...,"Rua Maria De Lourdes Kudri, 127, Barreirinha -...","Aluguel R$ 700,00 Seguro Incêndio R$ 26,26Tota...","41 m², 2 quartos",Apartamento com 2 quartos no Bairro Barreirinh...,Barreirinha,Curitiba,...,NaN,26.26,NaN,NaN,Não,Não,Não,Não,Não,Não
360,Apolar,2024-02-18,Apartamento Residencial para Locação no Campo ...,https://www.apolar.com.br/alugar/curitiba/camp...,"Rua Íris Antônio Campos, 250, Campo Comprido -...","Aluguel R$ 700,00 Condomínio R$ 400,00 Seguro ...","54 m², 1 banheiro, 1 vaga, 2 quartos, pet",Apartamento com 2 quartos no Bairro Campo Comp...,Campo Comprido,Curitiba,...,400.00,26.26,NaN,NaN,Não,Não,Não,Não,Não,Não


In [32]:
BUCKET_NAME = 'busca-apartamentos-trusted'
FILE_NAME = f'2024-02-18 - apartamentos - apolar.csv'
TEMP_FILE = 'local.csv'
RAW_PATH = '/tmp/{}'.format(TEMP_FILE)

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

df[columns_selected].to_csv(RAW_PATH, index=False)
blob = bucket.blob(FILE_NAME)
blob.upload_from_filename(RAW_PATH)